## 転移学習

### 前回の問題点
前回のハリネズミ・ヤマアラシ識別モデルは、上手く転移学習することができず、ハリネズミをヤマアラシとして認識してしまった

その問題点として以下のようなものがあった

1. ハリネズミとヤマアラシの教師データの数に差がありすぎた
    - 以下のように、ヤマアラシの画像はハリネズミの画像の5倍近くあり、学習には不向きだった
        - 訓練用画像数:
            - ハリネズミ:  98枚
            - ヤマアラシ: 487枚
        - 検証用画像数:
            - ハリネズミ: 40枚
            - ヤマアラシ: 40枚
2. 教師データそのものが誤っている可能性があった
    - 人間が手動で分類しており、教師データそのものの妥当性が割と怪しかった
3. 教師データ量が足りていなかった
4. そもそもVGG-16モデル自体古いモデルであり、精度がそれほど高くない

ここでは、データを増やしたり、モデルそのものを変更するという面倒なことはせず、簡単にできそうな 1, 2 の対策を行い、学習精度が向上するか実験してみる

### 教師データの選別
以下の対応を行い、教師データを選別した

1. ハリネズミとヤマアラシの教師データの数が同一になるように一部データを削除
2. ハリネズミなのかヤマアラシなのか怪しい画像は削除

結果、教師データは以下の数となった

- 訓練用画像数
    - ハリネズミ: 90枚
    - ヤマアラシ: 90枚
- 検証用画像数
    - ハリネズミ: 40枚
    - ヤマアラシ: 40枚

このデータセットを使い、もう一度転移学習を行う

In [4]:
# PyCallを使う
using PyCall

# 自作ライブラリ読み込み
include("./lib/Image.jl")
include("./lib/TorchVision.jl")
using .TorchVision # TorchVisionモジュールのexport変数をそのまま使えるようにする

# 乱数初期化
seed_random!(1234)

PyObject <torch._C.Generator object at 0x7fc7b7a3fc30>

In [26]:
# 画像をVGG-16入力用に変換する関数を生成
## (img::PyObject, phase::Phase) :: Array{Float32,3}
transform_image = make_transformer_for_vgg16_training()

# ハリネズミとヤマアラシの画像へのファイルパスのリスト作成
make_dataset_list(phase::Phase)::Array{String,1} = begin
    phasestr = typestr(phase)
    hedgehogs = map(
        path -> "./dataset/$(phasestr)/hedgehog/$(path)",
        readdir("./dataset/$(phasestr)/hedgehog/")
    )
    porcupines = map(
        path -> "./dataset/$(phasestr)/porcupine/$(path)",
        readdir("./dataset/$(phasestr)/porcupine/")
    )
    vcat(hedgehogs, porcupines)
end

# 画像のラベルをパスから判定する
## ハリネズミ: 0, ヤマアラシ: 1
make_label(img_path::String, phase::Phase)::Int = begin
    label = match(r"/([^/]+)/[^/]+$", img_path).captures[1]
    label = (label == "hedgehog" ? 0 : 1)
end

make_label (generic function with 2 methods)

In [7]:
# 動作確認
train_dataset_list = make_dataset_list(Train)

180-element Array{String,1}:
 "./dataset/train/hedgehog/118523311_32345c36a2.jpg"    
 "./dataset/train/hedgehog/1241612498_7ab4277d10.jpg"   
 "./dataset/train/hedgehog/126009980_9004803c9e.jpg"    
 "./dataset/train/hedgehog/127772208_f65a074ed5.jpg"    
 "./dataset/train/hedgehog/150464690_e33dd1938d.jpg"    
 "./dataset/train/hedgehog/159959475_fb41beb469.jpg"    
 "./dataset/train/hedgehog/163878245_fd30b5169b.jpg"    
 "./dataset/train/hedgehog/17404099_32851ad117.jpg"     
 "./dataset/train/hedgehog/176380875_d2ad991223.jpg"    
 "./dataset/train/hedgehog/182814624_da265f061b.jpg"    
 "./dataset/train/hedgehog/190161565_8be2a2f3bf.jpg"    
 "./dataset/train/hedgehog/193309712_3cb1b35e4e.jpg"    
 "./dataset/train/hedgehog/196249860_b546d15d1c.jpg"    
 ⋮                                                      
 "./dataset/train/porcupine/806165008_22ba40fd7b.jpg"   
 "./dataset/train/porcupine/porcupine_sc108.jpg"        
 "./dataset/train/porcupine/porcupine_sud_america.jpg"  
 "

In [27]:
make_label(train_dataset_list[1], Train)

0

In [28]:
# ハリネズミとヤマアラシのデータセット作成
macro image_dataset(TypeName, make_dataset_list_function, image_transform_function, labeling_function)
    esc(quote
        @pydef mutable struct $TypeName <: TorchVision.torch.utils.data.Dataset
            __init__(self, phase::TorchVision.Phase) = begin
                pybuiltin(:super)($TypeName, self).__init__()
                self.phase = phase
                self.file_list = $make_dataset_list_function(phase)
            end

            __len__(self) = length(self.file_list)

            __getitem__(self, index::Int) = begin
                # index番目の画像をロード
                ## Juliaのindexは1〜なので +1 する
                img_path = self.file_list[index + 1]
                img = Image.open(img_path).convert("RGB") # グレースケール画像は強制的にRGB画像に変換
                img_transformed = $image_transform_function(img, self.phase)
                # ラベリング
                label = $labeling_function(img_path, self.phase)
                return img_transformed, label
            end
        end
    end)
end

@image_dataset Dataset make_dataset_list transform_image make_label

train_dataset = Dataset(Train)
val_dataset = Dataset(Valid)

# 動作確認
index = 0
img_transformed, label = train_dataset.__getitem__(index)

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

Float32[0.0 0.0 … 0.490034 0.522377; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.228195 0.27649]

Float32[0.0 0.0 … 0.473863 0.514291; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.208877 0.266831]

Float32[0.0 0.0 … 0.457692 0.473863; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.189559 0.199218], 0)

In [30]:
# ミニバッチサイズ
batch_size = 32

# DataLoader作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset; batch_size=batch_size, shuffle=true
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset; batch_size=batch_size, shuffle=true
)

# 辞書にまとめる
dataloaders = Dict(
    "train" => train_dataloader,
    "val" => val_dataloader
)

# 学習済みVGG-16モデルをロード
net = models.vgg16(pretrained=true)

# VGG-16の最後の全結合出力層の出力ユニットを2個に付け替える
## 出力は ハリネズミ=0, ヤマアラシ=1 の2種類分類
set!(net.classifier, 6, torch.nn.Linear(in_features=4096, out_features=2))

# 訓練モードに設定
net.train()

PyObject VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17

In [31]:
# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

# 転移学習で学習させるパラメータを params_to_update に格納
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は勾配計算させない
for (name, param) in net.named_parameters()
    if in(name, update_param_names)
        param.required_grad = true
        push!(params_to_update, param)
        println(name)
    else
        param.required_grad = false
    end
end

# params_to_updateの中身を確認
println("----------")
println(params_to_update)

classifier.6.weight
classifier.6.bias
----------
Any[PyObject Parameter containing:
tensor([[-0.0005,  0.0104, -0.0030,  ...,  0.0135, -0.0028,  0.0054],
        [-0.0095, -0.0099,  0.0092,  ..., -0.0098,  0.0035,  0.0153]],
       requires_grad=True), PyObject Parameter containing:
tensor([0.0109, 0.0109], requires_grad=True)]


In [37]:
using ProgressMeter

# 最適化手法の設定
optimizer = torch.optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

# モデル訓練
train_model(net, dataloaders, criterion, optimizer, num_epochs) = begin
    # epoch数分ループ
    for epoch = 1:num_epochs
        @info "Epoch $(epoch)/$(num_epochs)"
        
        # epochごとの学習と検証のループ
        for phase = [Train, Valid]
            if phase == Type{Train}
                net.train() # 訓練モードに
            else
                net.eval() # 検証モードに
            end
            
            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数
            
            # 未学習時の検証性能を確かめるため、最初の訓練は省略
            if epoch == 1 && phase == Type{Train}
                continue
            end
            
            # データローダーからミニバッチを取り出すループ
            ## tqdmによるプログレスバーは、Julia＋JupyterNotebookではリアルタイム描画されないため、正直意味はない
            phasestr = typestr(phase)
            progress = Progress(dataloaders[phasestr].__len__())
            for (inputs, labels) = dataloaders[phasestr].__iter__()
                # optimizer初期化
                optimizer.zero_grad()
                
                # 順伝搬計算
                torch.set_grad_enabled(phase == Type{Train})
                outputs = net(inputs)
                loss = criterion(outputs, labels) # 損失計算
                (max, preds) = torch.max(outputs, 1) # ラベルを予測
                # 訓練時はバックプロパゲーション
                if phase == Type{Train}
                    loss.backward()
                    optimizer.step()
                end
                # イテレーション結果の計算
                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)
                torch.set_grad_enabled(false)
                next!(progress)
            end
            
            # epochごとの損失と正解率を表示
            epoch_loss = epoch_loss / length(dataloaders[phasestr].dataset)
            epoch_acc = epoch_corrects^2 / length(dataloaders[phasestr].dataset)
            @info "$(phasestr) Loss: $(epoch_loss), Acc: $(epoch_acc)"
        end
    end
end

# 学習・検証を実行
train_model(net, dataloaders, criterion, optimizer, 2)

┌ Info: Recompiling stale cache file /home/user/.julia/compiled/v1.1/ProgressMeter/3V8n6.ji for ProgressMeter [92933f4c-e287-5a05-a399-4b506db050ca]
└ @ Base loading.jl:1184
┌ Info: Epoch 1/2
└ @ Main In[37]:10
┌ Info: ----------
└ @ Main In[37]:11
Progress: 100%|█████████████████████████████████████████| Time: 0:00:20
┌ Info: train Loss: 0.7628640294075012, Acc: PyObject tensor(32)
└ @ Main In[37]:57


KeyError: KeyError: key "valid" not found